# QUICK MODEL BUILD

# IMPORT LIB

In [2]:
import sys  
sys.path.insert(0, '/Users/terra-016/Downloads/slik_python_package/')

In [3]:
from slik import pipeline as ppl

In [4]:
from slik import preprocessing as pp

In [4]:
import mlflow
import catboost
from sklearn.linear_model import LogisticRegression

In [5]:
from slik import loadfile as lf

In [6]:
from slik import utils as ut

In [18]:
train = lf.read_file('/Users/terra-016/Downloads/Installs and Download.csv',nrows=50000)


CSV file read sucessfully

Data has 27564 rows and 23 columns


In [19]:
new_df = train[train.event_type!='af_complete_registration']

# Model Selection
Choose an Algorithm of your choice.

Pass a dict of params if you want. The dictionary keys should either be model, fit or hyperparameters.


In [20]:
params = {
    'model':{
        'random_state':0, 
        'max_iter':1000 
    }}
alg = LogisticRegression()

In [21]:
from sklearn.ensemble import RandomForestClassifier
params = {
    'model':{
        'random_state':42, 
        'n_estimators':200 
    }}
alg = RandomForestClassifier()

In [22]:
params = {
      'model': {
          'iterations':2000, 
          "depth" :6, 
          "learning_rate":0.1, 
          "l2_leaf_reg": 10, 
          "loss_function":'Logloss',
          "eval_metric":'AUC'
      },

      'fit': {
        'early_stopping_rounds': 6,
        'logging_level': 'Silent',         
      },
    
        'hyperparameters':{'C': [0.1, 1.0, 10, 100],
               'solver': ['liblinear','newton-cg']
                          }
    }

model_params = params['model']
# fit_params = params['fit']

alg = catboost.CatBoostClassifier()

In [23]:
param= {'C': [0.1, 1.0, 10, 100],
               'solver': ['liblinear','newton-cg']}

# BUILDING PIPELINE

Basic understanding of pipelines

Build steps for numerical and categorical transformers

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
# Applying SimpleImputer and will search for different scalers using GridSearchCV
# numerical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())])
numerical_transformer = Pipeline(steps=[('scaler', StandardScaler())])
# categorical_transformer= Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='Missing'))])

In [25]:
# Applying SimpleImputer and then OneHotEncoder
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore',sparse = False))])

In [26]:
ppl.build_data_pipeline(new_df,clean_data=True,target_column='event_type',id_column='msisdn',display_inline=False,project_path='./InstallDownload/',\
                        numerical_transformer = numerical_transformer, categorical_transformer = categorical_transformer,\
                        pca=False,algorithm=alg,select_columns=['msisdn','sms_cost', 'spend_voice', 'spend_data', 'customer_class', \
                                                                'age', 'gender','location_region','location_state','device_manufacturer',\
                                                                'device_model','event_type'],\
                              params=params,hashing=False,nrows=7000)



--------------- Loading clean data ---------------


Clean data exists in ./InstallDownload/data/train_data.pkl



Use clean data:(y/n) 
 n



--------------- Identifying columns present in the data ---------------

Target column is transformed_event_type. Attribute in target column incldes:
[0, 1]

Features with high cardinality:['device_manufacturer', 'device_model']


--------------- Saving Attributes in Project path ---------------


Data columns successfully identified and attributes are stored in InstallDownload/data/metadata


--------------- Metric Performance ---------------


Metric performance on test data
{'accuracy': 0.8269689737470167, 'precision': 0.125, 'recall': 0.012345679012345678, 'f1': 0.02247191011235955}



confusion matrix
[[2074   35]
 [ 400    5]]


Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('data_transformer',
                                  ColumnTransformer(transformers=[('numerical',
                                                                   Pipeline(steps=[('scaler',
                                                                                    StandardScaler())]),
                                                                   ['sms_cost',
                                                                    'spend_voice',
                                                                    'spend_data',
                                                                    'customer_class',
                                                                    'age',
                                                                    'transformed_gender']),
                                                                  ('categorical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(fill_value='Missing',
                                                                                                  strategy='constant')),
                                                                                   ('onehot',
                                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                                  sparse=False))]),
                                                                   ['location_region',
                                                                    'location_state',
                                                                    'device_manufacturer',
                                                                    'device_model',
                                                                    'binned_age'])]))])),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x7ff249a6d9d0>)])

In [18]:
from slik import utils as ut
ut.load_pickle('./InstallDownload/data/validation_data.pkl')

,msisdn,timeline,timestamp,event_type,ad_text,spend_total,spend_vas,sms_cost,spend_voice,spend_data,...,timestampIs_month_end,timestampIs_month_start,timestampIs_quarter_end,timestampHour,timestampMinute,timestampIs_quarter_start,timestampIs_year_end,timestampIs_year_start,timestampElapsed,transformed_gender
0,af81bb96f9ccd7ecc223495ba44efc70,1.561736e+12,2019-07-07 07:59:40.000,install,All your favorite songs in just one place! Dow...,7366.400000,0.000000,0.000000,0.000000,0.000000,...,False,False,False,7,59,False,False,False,1562486380,0
2,fa4bcfafd145868f0119bb89bd1b17db,1.564159e+12,2019-09-20 07:09:18.408,install,Stop running helter-skelter when you need mone...,3224.950000,315.000000,88.000000,2617.230000,60.000000,...,False,False,False,7,9,False,False,False,1568963358,0
3,da2ae0a034979823b27139bb9479a6d9,1.562234e+12,2019-07-07 10:46:07.000,install,Listen to the latest hit songs from your favor...,5360.216671,459.040738,86.083116,2576.900000,636.508923,...,False,False,False,10,46,False,False,False,1562496367,0
4,f41536082453e49ad8db7e2eb7365112,1.564159e+12,2019-09-17 13:36:03.466,install,Stop running helter-skelter when you need mone...,5360.216671,459.040738,192.000000,1636.140000,1000.000000,...,False,False,False,13,36,False,False,False,1568727363,0
5,e7fd275d9958fa5d0219f5f0c21104d6,1.564159e+12,2020-05-16 05:55:33.454,af_app_opened,Hurry and win 15GB of data. Download the ALAT ...,299.816720,0.000000,0.000000,0.000000,0.000000,...,False,False,False,5,55,False,False,False,1589608533,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,cd7486a23904e5c93bb7691a21905e57,1.564159e+12,2020-06-29 12:07:11.990,install,"Need to Invest, But No capital? Install the AL...",5360.216671,213.097732,43.897461,1611.332573,445.096367,...,False,False,False,12,7,False,False,False,1593432431,0
9996,6b2a1050505122ed8d8d1405263a2586,1.564159e+12,2019-08-26 17:11:17.067,install,Claim your daily BONUS to get as high as 70K! ...,3817.303809,213.097732,43.897461,1611.332573,445.096367,...,False,False,False,17,11,False,False,False,1566839477,0
9997,d5dd1b830c3144e73a3c80ec36565f41,1.564159e+12,2019-09-14 08:49:53.302,install,Stop running helter-skelter when you need mone...,5461.595000,0.000000,86.083116,3094.650000,145.000000,...,False,False,False,8,49,False,False,False,1568450993,0
9998,f0b590252c708f849d085342da4cf858,1.564159e+12,2019-09-14 11:25:14.821,install,Stop running helter-skelter when you need mone...,5360.216671,459.040738,86.083116,4867.950000,0.000000,...,False,False,False,11,25,False,False,False,1568460314,0


# PIPELINE TRANSFORM

In [16]:
ppl.pipeline_transform_predict(data='/Users/terra-016/Downloads/Installs and Download.csv',
                  project_path='./InstallDownload/',model_path='./InstallDownload/model/CatBoostClassifier.pkl')

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
y = dataframe['transformed_event_type'].loc[list(train_df.index)]

In [17]:
ppl.evaluate_model('./InstallDownload/model/CatBoostClassifier.pkl',new_df,project_path='./InstallDownload/')


--------------- Metric Performance ---------------

{'accuracy': 0.7613532110091743, 'precision': 0.6614626129827444, 'recall': 0.3253839935327405, 'f1': 0.4361961528041181}

--------------- Saving Plots ---------------

Logged ./InstallDownload/plots/scores.png


/Users/terra-016/Downloads/slik_python_package/slik/pipeline.py:555: UserWarning: Transformer scaler (type StandardScaler) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "
/Users/terra-016/Downloads/slik_python_package/slik/pipeline.py:555: UserWarning: Transformer imputer (type SimpleImputer) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "


Logged ./InstallDownload/plots/feature_importance.png
Logged ./InstallDownload/plots/confusion_matrix.png
Logged ./InstallDownload/plots/roc_curve.png
Logged ./InstallDownload/plots/pr_curve.png
